<a href="https://colab.research.google.com/github/brandnewmin/brandnewm/blob/master/dacon3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)
%matplotlib inline

In [2]:
import missingno as msno #결측값보이기

In [3]:
pd.options.display.float_format='{:.2f}'.format

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/My Drive/data/train.csv')
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,nan,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,nan,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,nan,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,nan,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,nan,0,0


In [6]:
def splite_date(date):
  return date.split("-")

In [7]:
train['year'], train['month'], train['day'] = zip(*train['date'].apply(lambda x:splite_date(x)))

In [8]:
train['month'] = train['month'].astype(int) #무슨 차이일가 흠
#이게 왜 필요할까?
type(train['month'])

pandas.core.series.Series

In [9]:
train=train.fillna(1)

In [10]:
train['total_amount'] = train['amount'] * train['installments']

In [11]:
#dacon2랑 비교하려고 수정 안 함
fifth = train[['store_id','card_id','days_of_week','holyday','year','month','day','total_amount']].groupby(['store_id','card_id'], as_index=False).sum()
fifth.head()

,store_id,card_id,days_of_week,holyday,month,total_amount
0,0,000133a3c2,4,0,15,148.00
1,0,000744fa2e,5,0,9,55.00
2,0,000931da3a,0,0,11,38.00
3,0,0009529600,24,0,54,211.00
4,0,000abf87bd,38,1,76,3123.00


## 의문점

In [12]:
###################
#store_id별 총액을 알고 싶은거지
#cardid 구분이 왜 필요했는가? 그러게 왜 필요했지??
#시간별 총액 변하는 추이를 알고 싶었으니깐, month 구분별도 필요함

In [13]:
#cardid구분이 있는 경우
train_newgr=train[['store_id','card_id','days_of_week','holyday','year','month','day','total_amount']].groupby(['store_id','card_id','days_of_week','holyday','year','month','day'], as_index=False).sum()
train_newgr.head()
#fifth와 다른 점 : month day year 등이 합쳐지지 않음 -> 기대 : 월별 그래프 확인 가능

,store_id,card_id,days_of_week,holyday,year,month,day,total_amount
0,0,000133a3c2,1,0,2017,5,16,87.00
1,0,000133a3c2,2,0,2017,5,17,61.00
2,0,000744fa2e,5,0,2017,9,23,55.00
3,0,000931da3a,0,0,2017,11,06,38.00
4,0,0009529600,1,0,2017,9,05,38.00


In [14]:
train_newgr.shape #fifth와 유사한 데이터 : 행에서 차이가 있을 가능성 : 월별로도 그룹을 했기 때문에, store_id card_id 다음 가지가 또 있음 / 열 당연히 늘어남

(3165601, 8)

In [15]:
train_newgr['total_amount'].describe()
#train_newgr[train_newgr['total_amount'] >= 0]

count   3165601.00
mean        184.90
std        2393.58
min     -750000.00
25%          35.00
50%          80.00
75%         155.00
max     1198800.00
Name: total_amount, dtype: float64

In [16]:
#train_newgr[train_newgr['total_amount'] <0].value_counts()

In [17]:
newgr=train_newgr[train_newgr['total_amount'] >= 0]
newgr.head()

,store_id,card_id,days_of_week,holyday,year,month,day,total_amount
0,0,000133a3c2,1,0,2017,5,16,87.00
1,0,000133a3c2,2,0,2017,5,17,61.00
2,0,000744fa2e,5,0,2017,9,23,55.00
3,0,000931da3a,0,0,2017,11,06,38.00
4,0,0009529600,1,0,2017,9,05,38.00


In [18]:
newgr.shape #역시 train1과 행 같을 수 없음

(3163741, 8)

In [19]:
newgr['total_amount'].value_counts()

30.00      74841
35.00      68807
40.00      63663
80.00      62408
22.00      61651
           ...  
4923.00        1
4915.00        1
4912.00        1
4910.00        1
4311.00        1
Name: total_amount, Length: 5696, dtype: int64

In [20]:
newgr['total_amount'].describe()

count   3163741.00
mean        188.54
std        2159.01
min           0.00
25%          35.00
50%          80.00
75%         156.00
max     1198800.00
Name: total_amount, dtype: float64

## 월간 평균

In [21]:
#storeid별 월별 매출액 평균
monthly_storeid=newgr[['store_id','month','total_amount']].groupby(['store_id','month'], as_index=False).mean()
monthly_storeid.head()

,store_id,month,total_amount
0,0,1,95.70
1,0,2,89.79
2,0,3,84.37
3,0,4,88.81
4,0,5,84.58


In [22]:
monthly_storeid['total_amount'].describe() #stordid별 월간 평균

count    17242.00
mean      1456.89
std       9211.33
min          0.00
25%        105.11
50%        201.40
75%        563.81
max     453319.33
Name: total_amount, dtype: float64

In [23]:
#라벨링 (75%이하 값 0)
#75%이상 값 1
monthly_storeid['label'] = [1 if a > 563.81 else 0 for a in monthly_storeid['total_amount']]
monthly_storeid.head()

,store_id,month,total_amount,label
0,0,1,95.70,0
1,0,2,89.79,0
2,0,3,84.37,0
3,0,4,88.81,0
4,0,5,84.58,0


In [24]:
#시각화하면 되나요?

In [25]:
#가끔 알 수 없는 마이너스 값때문에 시간을 제외햇는데, 시간을 제외하면 시간별 자료 추이를 못 보니깐 아무튼 이렇게 하는게 맞는거고
#6월까지 구매한 것들을 7월에 환불한 경우를 제외할 순 없게 되었는데 이걸 생각 속에서 지워야할까?

In [29]:
#month별
train2222 = train1[['store_id','month','total_amount']].groupby(['store_id','month'], as_index=False).mean()
train2222.head()

,store_id,month,total_amount
0,0,1,93.93
1,0,2,93.88
2,0,3,87.95
3,0,4,95.46
4,0,5,100.11


## ----

In [26]:
train1 = fifth[fifth['total_amount'] >= 0]
train1.head()

,store_id,card_id,days_of_week,holyday,month,total_amount
0,0,000133a3c2,4,0,15,148.00
1,0,000744fa2e,5,0,9,55.00
2,0,000931da3a,0,0,11,38.00
3,0,0009529600,24,0,54,211.00
4,0,000abf87bd,38,1,76,3123.00


In [27]:
train1['total_amount'].describe()

count   1797861.00
mean        325.58
std        4218.53
min           0.00
25%          60.00
50%         119.00
75%         240.00
max     1671700.00
Name: total_amount, dtype: float64

## train2 : storeid별 매출액 평균

In [28]:
train2 = train1[['store_id','total_amount']].groupby(['store_id'], as_index=False).mean()
train2.head()

,store_id,total_amount
0,0,427.86
1,1,94556.25
2,2,178.40
3,3,273.14
4,4,229.86


In [30]:
under_train2 = train2[train2['total_amount'] <= 1064.59] #75%위치값
under_train2.head()

,store_id,total_amount
0,0,427.86
2,2,178.40
3,3,273.14
4,4,229.86
6,6,743.87


In [31]:
over_train2 = train2[train2['total_amount'] > 1064.59]
over_train2.head()

,store_id,total_amount
1,1,94556.25
5,5,3009.67
7,7,1479.86
9,9,13531.11
10,10,1301.43


In [32]:
train2['group'] = [1 if a > 1064.59 else 0 for a in train2['total_amount']]
train2.head()

,store_id,total_amount,group
0,0,427.86,0
1,1,94556.25,1
2,2,178.40,0
3,3,273.14,0
4,4,229.86,0


## 1. 총 매출액 평균 기준 0 1 group
2. 월간 평균 매출액 기준 0 1 group
3. 계절평 평균 매출액 기준 0 1 group

In [43]:
#맞는지 모르겠지만 일단 합시다
train2.head()
#print(train2.shape)

,store_id,total_amount,group
0,0,427.86,0
1,1,94556.25,1
2,2,178.40,0
3,3,273.14,0
4,4,229.86,0


In [ ]:
#매출액을 기준으로 75%위치한 값을 나눴는데 그 store 개수가 얼마인지 확인하면 좋겠군

## 2. 월간 평균 매출액 기준 0 1 label

In [44]:
monthly_storeid.head()

,store_id,month,total_amount,label
0,0,1,95.70,0
1,0,2,89.79,0
2,0,3,84.37,0
3,0,4,88.81,0
4,0,5,84.58,0


In [46]:
monthly_storeid['total_amount'].value_counts()

0.00       49
1500.00    39
1250.00    33
100.00     30
150.00     30
           ..
60.33       1
28.65       1
129.90      1
426.58      1
181.56      1
Name: total_amount, Length: 15346, dtype: int64

## 3. 계절별 평균 매출액 기준 0 1 group

In [68]:
season = []
for month in monthly_storeid['month']:
  if month < 3 :
    season.append(1) #겨울
  elif month < 7:
    season.append(2) #봄
  elif month < 9:
    season.append(3) #여름
  elif month < 12: season.append(4) #가을
  else : season.append(1)

monthly_storeid['season'] = season

In [69]:
monthly_storeid.head()

,store_id,month,total_amount,label,season
0,0,1,95.70,0,1
1,0,2,89.79,0,1
2,0,3,84.37,0,2
3,0,4,88.81,0,2
4,0,5,84.58,0,2


In [70]:
monthly_storeid['season'].value_counts()

2    6460
1    4190
4    3787
3    2805
Name: season, dtype: int64

In [71]:
#월별 평균매출액 (꾸준한 평균매출액을 보여주는 자료라고 할 수 있지 않나? total_amount 평균과 크게 다를 것 같진 않은데!)
pd.crosstab(monthly_storeid['label'],monthly_storeid['season'],margins=True).style.background_gradient(cmap='summer')
#지금 이 자료 되게 의미없네, season별 평균 매출액 높은 것들끼리 나눈 걸로 보는 것도 아니고 label대로 나누고 season은 이렇게 냅두면 이상하지 않나?

season,1,2,3,4,All
label,,,,,
0,3121,4869,2103,2838,12931
1,1069,1591,702,949,4311
All,4190,6460,2805,3787,17242


In [72]:
pd.crosstab(monthly_storeid['label'], train2['group'], margins=True).style.background_gradient(cmap='winter')
#총매출액 평균으로 0 1 나눈 그룹과 월별 평균 매출액 0 1 라벨 수

group,0,1,All
label,,,
0,1021,356,1377
1,310,88,398
All,1331,444,1775


In [ ]:
#계절별

In [ ]:
#이용자 수가 많으면 소비자 폭이 넓다는 거랑 거래수가 많으면 자주 접하기 쉬운 곳 vs 이용자수가 한정적이거나 거래수가 적은데 거래비용이 크다면 명품관 예상?

In [48]:
#monthly_storeid.value_counts()

## card_id별 소비액 평균

In [33]:
train3 = train1[['card_id','total_amount']].groupby(['card_id'], as_index=False).mean()
train3.head()

,card_id,total_amount
0,000009e765,10.00
1,00000f9eaa,110.33
2,0000134335,90.00
3,000024f480,80.00
4,000036a26b,720.00


In [34]:
#month 추가
train3_month = train1[['card_id','total_amount','month']].groupby(['card_id','month'], as_index=False).mean()
train3_month.head()

,card_id,month,total_amount
0,000009e765,7,10.00
1,00000f9eaa,3,123.00
2,00000f9eaa,10,100.00
3,00000f9eaa,24,108.00
4,0000134335,5,90.00


In [35]:
quarter = []
for month in train3_month['month'] :
  if month < 4:
    quarter.append(1)
  elif month < 7:
    quarter.append(2)
  elif month < 10:
    quarter.append(3)
  else : quarter.append(4)

train3_month['quarter'] = quarter

In [36]:
train3_month.head()

,card_id,month,total_amount,quarter
0,000009e765,7,10.00,3
1,00000f9eaa,3,123.00,1
2,00000f9eaa,10,100.00,4
3,00000f9eaa,24,108.00,4
4,0000134335,5,90.00,2


In [37]:
train3.describe()

,total_amount
count,922519.00
mean,338.57
std,3150.77
min,0.00
25%,75.00
50%,138.00
75%,250.00
max,1080000.00


In [38]:
under_train3 = train3[train3['total_amount'] <= 250] #75%위치값
under_train3.head()

,card_id,total_amount
0,000009e765,10.00
1,00000f9eaa,110.33
2,0000134335,90.00
3,000024f480,80.00
6,00003cb9a7,85.00


In [39]:
over_train3 = train3[train3['total_amount'] > 250]
over_train3.head()

,card_id,total_amount
4,000036a26b,720.00
5,00003c84e3,255.00
9,00008a3f32,335.00
12,0000aee7ef,360.00
13,0000d054a6,423.50


In [40]:
train3['group'] = [1 if a > 250 else 0 for a in train3['total_amount']]
train3.head()

,card_id,total_amount,group
0,000009e765,10.00,0
1,00000f9eaa,110.33,0
2,0000134335,90.00,0
3,000024f480,80.00,0
4,000036a26b,720.00,1


## 시각화